In [1]:
import datetime
import os
import gc
import numpy as np
import tensorflow as tf
from PIL import Image
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras import layers
from keras import models
from keras import optimizers
from keras import backend as K
import matplotlib.pyplot as plt
import warnings
%matplotlib inline
warnings.filterwarnings('ignore')

In [2]:
image_root_dir = 'E:\LargeDatasets\SAR-Ocean-Images\GeoTIFF\OrganisationForModel'
train_dir = f'{image_root_dir}\\train'
val_dir = f'{image_root_dir}\\val'
test_dir = f'{image_root_dir}\\test'

In [3]:
batch_size = 8

train_generator = ImageDataGenerator().flow_from_directory(
        train_dir, class_mode = 'categorical', color_mode = 'grayscale',
        target_size=(540, 490), batch_size= batch_size)

val_generator = ImageDataGenerator().flow_from_directory(
        val_dir, class_mode = 'categorical', color_mode = 'grayscale',
        target_size=(540, 490), batch_size = batch_size)

test_generator = ImageDataGenerator().flow_from_directory(
        test_dir, class_mode = 'categorical', color_mode = 'grayscale',
        target_size=(540, 490), batch_size = batch_size) 

Found 30041 images belonging to 10 classes.
Found 3756 images belonging to 10 classes.
Found 3756 images belonging to 10 classes.


In [5]:
image_path = 'E:\LargeDatasets\SAR-Ocean-Images\GeoTIFF\OrganisationForModel\\train\H\s1a-wv1-slc-vv-20160107t131721-20160107t131724-009388-00d975-029.tiff'
test_image = load_img(image_path, target_size = (540,490))

In [6]:
img_to_array(test_image)

array([[[255., 255., 255.],
        [255., 255., 255.],
        [255., 255., 255.],
        ...,
        [255., 255., 255.],
        [255., 255., 255.],
        [255., 255., 255.]],

       [[255., 255., 255.],
        [255., 255., 255.],
        [255., 255., 255.],
        ...,
        [255., 255., 255.],
        [255., 255., 255.],
        [255., 255., 255.]],

       [[255., 255., 255.],
        [255., 255., 255.],
        [255., 255., 255.],
        ...,
        [255., 255., 255.],
        [255., 255., 255.],
        [255., 255., 255.]],

       ...,

       [[255., 255., 255.],
        [255., 255., 255.],
        [255., 255., 255.],
        ...,
        [255., 255., 255.],
        [255., 255., 255.],
        [255., 255., 255.]],

       [[255., 255., 255.],
        [255., 255., 255.],
        [255., 255., 255.],
        ...,
        [255., 255., 255.],
        [255., 255., 255.],
        [255., 255., 255.]],

       [[255., 255., 255.],
        [255., 255., 255.],
        [255., 2

In [4]:
train_generator[0]

(array([[[[ 5208.],
          [ 8006.],
          [ 7369.],
          ...,
          [ 6709.],
          [ 5769.],
          [ 6162.]],
 
         [[16062.],
          [14882.],
          [14418.],
          ...,
          [12528.],
          [14049.],
          [14126.]],
 
         [[14554.],
          [12043.],
          [11189.],
          ...,
          [13178.],
          [13861.],
          [14161.]],
 
         ...,
 
         [[12395.],
          [17721.],
          [15952.],
          ...,
          [13826.],
          [13790.],
          [12741.]],
 
         [[13593.],
          [14812.],
          [14933.],
          ...,
          [11751.],
          [13229.],
          [13235.]],
 
         [[ 4731.],
          [ 4088.],
          [ 3691.],
          ...,
          [ 4041.],
          [ 3023.],
          [ 4032.]]],
 
 
        [[[ 7927.],
          [ 6528.],
          [ 7054.],
          ...,
          [ 7091.],
          [ 6398.],
          [ 7079.]],
 
         [[1516

In [5]:
model = models.Sequential()
model.add(layers.Conv2D(64, (3, 3), activation='relu',
                        input_shape=(540, 490, 1)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2),padding = 'same'))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2),padding = 'same'))
model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2),padding = 'same'))
model.add(layers.Conv2D(512, (3, 3), activation='relu'))
model.add(layers.Conv2D(512, (3, 3), activation='relu'))
model.add(layers.Conv2D(512, (3, 3), activation='relu'))
model.add(layers.Conv2D(512, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2),padding = 'same'))
model.add(layers.Conv2D(512, (3, 3), activation='relu'))
model.add(layers.Conv2D(512, (3, 3), activation='relu'))
model.add(layers.Conv2D(512, (3, 3), activation='relu'))
model.add(layers.Conv2D(512, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2),padding = 'same'))
model.add(layers.Flatten())
model.add(layers.Dense(1000, activation='relu'))
model.add(layers.Dense(200, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

In [6]:
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=.0001),
              metrics=['acc'])

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 538, 488, 64)      640       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 536, 486, 64)      36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 268, 243, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 266, 241, 128)     73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 264, 239, 128)     147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 132, 120, 128)     0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 130, 118, 256)     2

In [8]:
history = model.fit(train_generator,
                        steps_per_epoch= 30041//batch_size,
                        epochs=1,
                        validation_data=val_generator,
                        validation_steps= 3756//batch_size)

3755/3755 [==============================] - 1676s 446ms/step - loss: 1.9287 - acc: 0.4291 - val_loss: 1.0188 - val_acc: 0.6346


In [5]:
def instantiate_model():
    model = models.Sequential()
    model.add(layers.Conv2D(64, (3, 3), activation='relu',
                            input_shape=(540, 490, 3)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2),padding = 'same'))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2),padding = 'same'))
    model.add(layers.Conv2D(256, (3, 3), activation='relu'))
    model.add(layers.Conv2D(256, (3, 3), activation='relu'))
    model.add(layers.Conv2D(256, (3, 3), activation='relu'))
    model.add(layers.Conv2D(256, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2),padding = 'same'))
    model.add(layers.Conv2D(512, (3, 3), activation='relu'))
    model.add(layers.Conv2D(512, (3, 3), activation='relu'))
    model.add(layers.Conv2D(512, (3, 3), activation='relu'))
    model.add(layers.Conv2D(512, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2),padding = 'same'))
    model.add(layers.Conv2D(512, (3, 3), activation='relu'))
    model.add(layers.Conv2D(512, (3, 3), activation='relu'))
    model.add(layers.Conv2D(512, (3, 3), activation='relu'))
    model.add(layers.Conv2D(512, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2),padding = 'same'))
    model.add(layers.Flatten())
    model.add(layers.Dense(1000, activation='relu'))
    model.add(layers.Dense(200, activation='relu'))
    model.add(layers.Dense(10, activation='softmax'))
    return model

In [6]:
best_acc = 0
for i in range(36):
    model = instantiate_model()
    model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=.0001),
              metrics=['acc'])
    history = model.fit(train_generator,
                        steps_per_epoch= 30041//batch_size,
                        epochs=1,
                        validation_data=val_generator,
                        validation_steps= 3756//batch_size,
                        verbose = 2)
    acc = history.history['acc'][-1]
    if best_acc < acc:
        best_acc = acc
        print(f'New best accuracy: {best_acc}')
        model.save('.\SavedModels\iterCNN', overwrite = True)
    del model
    del history
    tf.keras.backend.clear_session()
    for i in range (100):
        gc.collect()

6008/6008 - 1742s - loss: 2.2701 - acc: 0.1257 - val_loss: 2.2293 - val_acc: 0.1305
New best accuracy: 0.12571580708026886
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: .\SavedModels\iterCNN\assets
6008/6008 - 1753s - loss: 2.2670 - acc: 0.1302 - val_loss: 2.2303 - val_acc: 0.1278
New best accuracy: 0.13017712533473969
INFO:tensorflow:Assets written to: .\SavedModels\iterCNN\assets
6008/6008 - 1735s - loss: 2.2520 - acc: 0.1277 - val_loss: 2.2302 - val_acc: 0.1278
6008/6008 - 1733s - loss: 2.2573 - acc: 0.1234 - val_loss: 2.2306 - val_acc: 0.1305
6008/6008 - 1733s - loss: 2.2527 - acc: 0.1254 - val_loss: 2.2319 - val_acc: 0.1254
6008/6008 - 1737s - loss: 2.2578 - acc: 0.1273 - val_loss: 2.2318 - val_acc: 0.1278
6008/6008 - 1735s - loss: 2.2591 - acc: 0.1253 - val_los

KeyboardInterrupt: 